#Backpropagation

In [1]:
import numpy as np
import cv2
import pandas as pd
from sklearn.metrics import accuracy_score
from lib.ReadFiles import read_csv
from lib.FaceDetection import find_faces
from lib.Prepocessing import image_equalizer
from lib.plot import *
def reshape_dataset(X, y):
    images = [np.array(dx) for dx in X]
    images = np.asarray(images, dtype=np.uint8)
    labels = np.array(y).tolist()
    return images, labels
	
emotions = ["Marah", "Jijik", "Takut",
            "Senang", "Sedih", "Terkejut", "Biasa saja"]  # inisialisasi index dan emosi

In [2]:
# _x, _y, _u = read_csv("input/fer2013_small.csv")
# print(np.array(_x).shape)

# x_new, y_new, u_new = [], [], []
# for i, pixel in enumerate(_x):
# 	for norm, (x,y,w,h) in find_faces(np.array(pixel, dtype=np.uint8).reshape(48, 48)):
# 		if norm.size >0:
# 			x_new.append(pixel)
# 			y_new.append(_y[i])
# 			u_new.append(_u[i])
# print(np.array(x_new).shape)

# x_, y_ = image_equalizer(x_new, y_new)

In [3]:
# dataframe = pd.DataFrame({"emotion": y_, "pixels": x_, "usage": u_new})

In [4]:
# _x_test, _y_test = dataframe.loc[dataframe["usage"].str.endswith("Test") ,"pixels"], dataframe.loc[dataframe["usage"].str.endswith("Test"), "emotion"]
# x_test, y_test = reshape_dataset(_x_test, _y_test)
# plot_figure(x_test, y_test)

In [5]:
# membaca model dari trainig fihserface
import joblib
fisher = joblib.load("model/fisherface_training.pkl")

In [6]:
# f_pred = []
# f_extract = []
# for i in range(len(x_test)):
# 	_p = fisher.predict(x_test[i]) #prediksi fisherface
# 	_q = fisher.extract(x_test[i]) #ekstrasi fitur fisherface
# 	f_pred.append(_p)
# 	f_extract.append(_q)

# print(accuracy_score(y_test, f_pred))
# cm_plot(y_test, f_pred, 'Test Fisherface', True)


In [7]:
# from doctest import OutputChecker
# import tensorflow as tf
# import numpy as np
# import matplotlib.pyplot as plt
# from tensorflow.keras.models import Sequential, save_model, load_model
# from tensorflow.keras.utils import to_categorical
# from tensorflow.keras.layers import Activation, Dense, Input
# from tensorflow.keras.optimizers import SGD
# from sklearn.metrics import confusion_matrix

# def normalize_data(x,y):
#     x = np.asarray(x, dtype=np.float32).reshape(len(x), -1)
#     y =  to_categorical(y)
#     return x,y

# class ModelBackprop(object):
# 	def __init__(self, x=None, y=None, n_hidden=0, lr=0, path = None):
# 		self.lr = lr
# 		self.n_output = 7
# 		self.n_epoch = 1000
# 		self.n_hidden = n_hidden
# 		self.model = Sequential()
# 		self.sgd = SGD(learning_rate=lr)
		
# 		self.init()

# 		if x is not None and y is not None:
# 			x, y = normalize_data(x, y)
# 			self.compute(x, y)
# 		if path is not None:
# 			self.saved(path)

# 	def init(self):
# 		self.model.add(Dense(self.n_hidden, input_shape=(6,)))
# 		self.model.add(Activation("sigmoid"))
# 		self.model.add(Dense(self.n_output))
# 		self.model.add(Activation("sigmoid"))

# 	def compute(self, x, y):
# 		self.model.compile(loss="MSE", optimizer=self.sgd, metrics=['accuracy'])
# 		self.history = self.model.fit(x, y, epochs=self.n_epoch, batch_size=3, verbose=0)
	
# 	def predict(self,x):
# 		x = np.asarray(x, dtype=np.float32).reshape(len(x), -1)
# 		return self.model.predict(x)

# 	def saved(self, path):
# 		to_json = self.model.to_json()
# 		with open("model.json", 'w') as json_file:
# 			json_file.write(to_json)
# 		self.model.save_weights(path)

In [8]:
# bpn = ModelBackprop(x = f_extract, y = y_test, n_hidden=128, lr=0.25, path='model/weight.h5')

In [9]:
# history_plot(bpn.history, 'Test History', False)

In [10]:
from tensorflow.keras.models import model_from_json
#load model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model/weight.h5")
print("Loaded model from disk")

Loaded model from disk


In [15]:
import cv2
import numpy as np
WIDTH = 48
HEIGHT = 48
x=None
y=None

full_size_image = cv2.imread("test.png")
print("Image Loaded")
gray=cv2.cvtColor(full_size_image,cv2.COLOR_RGB2GRAY)
face = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
faces = face.detectMultiScale(gray, 1.1  , 6, minSize=(24,24))

#detecting facesq
for (x, y, w, h) in faces:
		roi_gray = gray[y:y + h, x:x + w]
		cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray, (48, 48)), -1), 0)
		cv2.normalize(cropped_img, cropped_img, alpha=0, beta=1, norm_type=cv2.NORM_L2, dtype=cv2.CV_32F)
		cv2.rectangle(full_size_image, (x, y), (x + w, y + h), (0, 255, 0), 1)
        #predicting the emotion
		xhat = fisher.extract(cropped_img)
		yhat= loaded_model.predict(xhat)
		cv2.putText(full_size_image, emotions[int(np.argmax(yhat))], (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 1, cv2.LINE_AA)
		print("Emotion: "+emotions[int(np.argmax(yhat))])

cv2.imshow('Emotion', full_size_image)
cv2.waitKey()

Image Loaded
Emotion: Senang


113

In [12]:
# cap = cv2.VideoCapture(0)
# while True:
# 	rval, vFrame = cap.read()
# 	vFrame = cv2.flip(vFrame, 1)
# 	faceCascade = cv2.CascadeClassifier('C:\Program Files\Python37\Lib\site-packages\cv2\data\haarcascade_frontalface_alt.xml')
# 	cv2image = cv2.cvtColor(vFrame, cv2.COLOR_RGB2BGR)
# 	cv2image = cv2.cvtColor(cv2image, cv2.COLOR_BGR2GRAY)
# 	faces = faceCascade.detectMultiScale(cv2image, scaleFactor=1.1, minNeighbors=6, minSize=(24,24))

# 	for (x,y, w, h) in faces:
# 		roi_gray = cv2image[y:y + h, x:x + w]
# 		cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray, (48, 48)), -1), 0)
# 		cv2.normalize(cropped_img, cropped_img, alpha=0, beta=1, norm_type=cv2.NORM_L2, dtype=cv2.CV_32F)
# 		cv2.rectangle(vFrame, (x, y), (x + w, y + h), (0, 255, 0), 1)
#         #predicting the emotion
# 		xhat = fisher.extract(cropped_img)
# 		yhat= loaded_model.predict(xhat)
# 		cv2.putText(vFrame, emotions[int(np.argmax(yhat))], (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 1, cv2.LINE_AA)
# 	cv2.imshow('Emotion', vFrame)
# 	if cv2.waitKey(30) & 0xff == ord('q'):
# 		break
# cap.release()
# cv2.destroyAllWindows()